## TensorFlow: A Short Introduction

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def get_weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    # TODO: Return weights
    return tf.Variable(tf.truncated_normal((n_features, n_labels)))


def get_biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    # TODO: Return biases
    return tf.Variable(tf.zeros(n_labels))


def linear(_input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    # TODO: Linear Function (xW + b)
    return tf.add(tf.matmul(_input, w), b)

In [2]:
def mnist_features_labels(n_labels):
    """
    Gets the first <n> labels from the MNIST dataset
    :param n_labels: Number of labels to use
    :return: Tuple of feature list and label list
    """
    mnist_features = []
    mnist_labels = []

    mnist = input_data.read_data_sets('./datasets/ud730/mnist', one_hot=True)

    # In order to make quizzes run faster, we're only looking at 10000 images
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):

        # Add features and labels if it's for the first <n>th labels
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])

    return mnist_features, mnist_labels

def run_lin_func_test():
    # Number of features (28*28 image is 784 features)
    n_features = 784
    # Number of labels
    n_labels = 3

    # Features and Labels
    features = tf.placeholder(tf.float32)
    labels = tf.placeholder(tf.float32)

    # Weights and Biases
    w = get_weights(n_features, n_labels)
    b = get_biases(n_labels)

    # Linear Function xW + b
    logits = linear(features, w, b)

    # Training data
    train_features, train_labels = mnist_features_labels(n_labels)

    with tf.Session() as session:
        # TODO: Initialize session variables
        session.run(tf.global_variables_initializer())

        # Softmax
        prediction = tf.nn.softmax(logits)

        # Cross entropy
        # This quantifies how far off the predictions were.
        # You'll learn more about this in future lessons.
        cross_entropy = -tf.reduce_sum(
            labels * tf.log(prediction), reduction_indices=1)

        # Training loss
        # You'll learn more about this in future lessons.
        loss = tf.reduce_mean(cross_entropy)

        # Rate at which the weights are changed
        # You'll learn more about this in future lessons.
        learning_rate = 0.08

        # Gradient Descent
        # This is the method used to train the model
        # You'll learn more about this in future lessons.
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

        # Run optimizer and get loss
        _, loss = session.run(
            [optimizer, loss],
            feed_dict={features: train_features,
                       labels: train_labels})
        
        return loss
    

print('Loss: {}'.format(run_lin_func_test()))

Extracting ./datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz
Loss: 6.51898193359375


In [3]:
# Softmax normalization in TF. 
def run_softmax():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)

    # TODO: Calculate the softmax of the logits
    softmax = tf.nn.softmax(logits)

    with tf.Session() as sess:
        # TODO: Feed in the logit data
        output = sess.run(softmax, feed_dict={logits: logit_data})

    return output

In [4]:
# Softmax normalization in TF. 
def run_softmax():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)

    # TODO: Calculate the softmax of the logits
    softmax = tf.nn.softmax(logits)

    with tf.Session() as sess:
        # TODO: Feed in the logit data
        output = sess.run(softmax, feed_dict={logits: logit_data})

    return output

print(run_softmax())

[ 0.65900117  0.24243298  0.09856589]


In [5]:
# OneHotEncoder using sklearn
from sklearn import preprocessing
def one_hot_encoding():
    # Example labels
    labels = np.array([1, 5, 3, 2, 1, 4, 2, 1, 3])

    # Create the encoder
    lb = preprocessing.LabelBinarizer()

    # Here the encoder finds the classes and assigns one-hot vectors
    lb.fit(labels)

    # And finally, transform the labels into one-hot encoded vectors
    return lb.transform(labels)

print(one_hot_encoding())

[[1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]]


In [6]:
# Cross Entropy in TensorFlow
def tf_cross_entropy():
    softmax_data = [0.7, 0.2, 0.1]
    one_hot_data = [1.0, 0.0, 0.0]

    softmax = tf.placeholder(tf.float32)
    one_hot = tf.placeholder(tf.float32)
    
    # TODO: Print cross entropy from session
    idx_mul = tf.multiply(one_hot, tf.log(softmax))
    cross_entropy = -1.0 * tf.reduce_sum(idx_mul)
    
    with tf.Session() as sess:
        # calc = -1.0 * tf.reduce_sum(tf.matmul(one_hot, softmax))
        output = sess.run(cross_entropy, feed_dict={one_hot:one_hot_data, softmax:softmax_data})
    
    return output

print(tf_cross_entropy())

0.356675


### Always worry about the calculation between large and small values:

In [26]:
def what_the_hell():
    base = 1e9
    a = base
    n_iter = int(1e6)
    for i in range(n_iter):
        a += + (1 / n_iter)
    return (a - base)

print(what_the_hell())

0.95367431640625


In [44]:
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    outout_batches = []
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches

from pprint import pprint

# 4 Samples of features
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]
# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

# PPrint prints data structures like 2d arrays, so they are easier to read
pprint(batches(3, example_features, example_labels))


def tf_test_batch():
    learning_rate = 0.001
    n_input = 784  # MNIST data input (img shape: 28*28)
    n_classes = 10  # MNIST total classes (0-9 digits)

    # Import MNIST data
    mnist = input_data.read_data_sets('./datasets/ud730/mnist', one_hot=True)

    # The features are already scaled and the data is shuffled
    train_features = mnist.train.images
    test_features = mnist.test.images

    train_labels = mnist.train.labels.astype(np.float32)
    test_labels = mnist.test.labels.astype(np.float32)

    # Features and Labels
    features = tf.placeholder(tf.float32, [None, n_input])
    labels = tf.placeholder(tf.float32, [None, n_classes])

    # Weights & bias
    weights = tf.Variable(tf.random_normal([n_input, n_classes]))
    bias = tf.Variable(tf.random_normal([n_classes]))

    # Logits - xW + b
    logits = tf.add(tf.matmul(features, weights), bias)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    # Calculate accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


    # TODO: Set batch size
    batch_size = 128
    assert batch_size is not None, 'You must set the batch size'

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)

        # TODO: Train optimizer on all batches
        for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
            sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})

        # Calculate accuracy for test dataset
        test_accuracy = sess.run(
            accuracy,
            feed_dict={features: test_features, labels: test_labels})
    
    return test_accuracy

print('Test Accuracy: {}'.format(tf_test_batch()))

[[[['F11', 'F12', 'F13', 'F14'],
   ['F21', 'F22', 'F23', 'F24'],
   ['F31', 'F32', 'F33', 'F34']],
  [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]],
 [[['F41', 'F42', 'F43', 'F44']], [['L41', 'L42']]]]
Extracting ./datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz
Test Accuracy: 0.1426999866962433


In [49]:
def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))
    

# def tf_test_batch_with_epochs():
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('./datasets/ud730/mnist', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 128
epochs = 10
learn_rate = 0.001

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})
        
        
print('Test Accuracy: {}'.format(test_accuracy))

Extracting ./datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting ./datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting ./datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting ./datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz
Epoch: 0    - Cost: 15.7     Valid Accuracy: 0.0972
Epoch: 1    - Cost: 14.5     Valid Accuracy: 0.102
Epoch: 2    - Cost: 13.5     Valid Accuracy: 0.108
Epoch: 3    - Cost: 12.7     Valid Accuracy: 0.118
Epoch: 4    - Cost: 12.0     Valid Accuracy: 0.13 
Epoch: 5    - Cost: 11.4     Valid Accuracy: 0.146
Epoch: 6    - Cost: 10.9     Valid Accuracy: 0.168
Epoch: 7    - Cost: 10.5     Valid Accuracy: 0.193
Epoch: 8    - Cost: 10.1     Valid Accuracy: 0.214
Epoch: 9    - Cost: 9.69     Valid Accuracy: 0.233
Test Accuracy: 0.24619996547698975
